# Halite challenge - advanced topics

We have seen how to solve the reinforcement learning problem of a single ship that has to optimize the collection of halite. We now want to extend the learning process to multiple ships. To do that we face two new kind of problems/tasks in addition to the one already solved for the single ship case. The first one is how to teach to the shipyard to spawn efficiently the ships. The second one is how to coordinate all the ships at each turn and also in the learning process.

#  Multi-agent framework

Two classes of agents: ship and shipyard. Only one instance of shipyard agent, many of the ship agent.

### Challenges:

1. Interface all the ship agents in order to make them learn withouth crashing into each other;
2. Define for each class of agents what is the (partial) state that they observe at each turn;
3. Shape the reward for multiple ships (one for all or one for each?) and for the shipyard (just a single reward for all the episode?);
4. Choose how to iteratively train the two classes whithout making confiusion that could ruin the training procedure, e.g. assigning a penalty to the shipyard for a fault of a ship agent.


# Ship class with tabular method

The simplest and maybe more straightforward way to generalize from one ship to many ships is to consider them as separate and independent entities, that share as little as possible between them (from the point of view of the state observed in the learning process) so that their learning is very similar to the single ship case. Then the trick is to use a single Q-value table that is shared by all of them, to which each ship will contribute by estimating some of the values from its direct experience; in this way it is required much less experience to train them. In fact the number of possible states of the system would grow exponentially if we were to consider as a state the union of the states of all ships and that would be infeasible even for two ships. For example for a single ship in a $7 \times 7$ map we have 142.884 possible states; consequently we would have $(142.844)^2 = 20.4\cdot10^9$ states. Considering that each of these states has to be multiplied for all the possible combined actions of the two ships (25) and requires 64 bits, i.e. 8 bytes, to be stored, the memory required to store the Q-value table is $4.08\cdot 10^{12}$ bytes = 4.08 Tb, that definetively doesn't fit any existing RAM.

## State representation

Assessed the fact that a shared state is infeasible, what we can do is to encode in a minimal way some the information that is necessary to a ship to avoid collisions. Since the maximum range of the composed motion of two ships is 2 cells and near a ship there can be multiple ships, we can choose to encode the positions of the other ships inside a rectangle of $5\times5$ relative to the position of the considered ship. But in this way we also include cells that are more than two moves away from the center (because diagonal moves aren't allowed).
In the grid below the `___` are the cells in the range of the two moves, instead the `***` are the ones out of range.

| / | 0| 1 | 2 | 3 | 4 |
| --- | --- | --- | --- | --- | --- |
|0    | *** | *** | ___ | *** | *** |
|1    | *** | ___ | ___ | ___ | *** |
|2    | ___ | ___ | ship | ___ | ___ |
|3    | *** | ___ | ___ | ___ | *** |
|4    | *** | *** | ___ | *** | *** |

All we need is the information about 13 out of the 25 cells. Each of these cells can be occupied (1) or empty (0), thus all the possible combinations are $2^{13} = 8192$. This results in a total number of state-actions of $5.850\cdot 10^9$ and 47 Gb of space. Again this is too much.

Thinking about the scope of this information, we can easily realize that is all about what are the dangerous directions and what are the safe ones. Since we are forced to compress this information, the minimal but still relevant information would be a single direction, the safest one. We can label dangerous a direction if taking an action in that direction has non-zero probability of colliding with another ship. If we have just one dangerous direction, the safest one is the opposite direction; if we have two, we can choose randomly one of the two safe directions; if we have three, only one direction remains and finally if all four direction are dangerous, the safest option is to remain still (this doesn't ensure the safety of the ship, but if all the ships follow the same optimal policy should work). So we can use a variable `safe_dir` $\in [0,4]$ to encode this information. This results in in a total number of state-actions of $3.57\cdot 10^6$ and 28.57 Mb of space. 

## Reward

Since we choose to use independent agents for all the ships, it makes sense to decompose the reward into the contributions of all the ships. The contribution of a ship is 0 if it doesn't deposit halite (and in that case it receives the baseline reward, e.g. -0.01) and is the halite stored divided by 1000 minus the baseline reward for the ship that actually has deposited the halite. In case that two ships crush one into the other give a reward of -1 to each of them (like if 1000 halite was lost, that is the cost of the ship). We can even add a second term to the penalty that estimates the potential gain that the ship would have brought in the future, but it can be a complicated estimate and -1 is still a very bad reward to experience. But then of course we reach a terminal state for those agents and the value of those terminal states aren't representable in the ship coordinates (because they are not even concieved as possible while the ship is alive), thus we must assign them a value, that could be the one that the ship would have received if it were stuck in that cell untill the end of the episode, i.e. the baseline reward (penalty of 0.01) times the number of turns left. 

# Shipyard class

Dealing with the shipyard is a completely different problem, for different reasons:

1. It has at disposal a binary choice (spawn ship or stay still);
2. It's not responsible for the absolute result of the episode, but its choices can influence it greatly;
3. Its choices are more affected by the current time of the episode than the ship ones (spawning a ship in the last few turns it's definitively a bad idea);
4. It receives feedback about its action with a much lower frequence than the ship agents, because the only metric to evaluate its policy is the final amount of halite collected and how it has changed through the episodes while keeping fixed the policy of the ships.

If we try to understand what are the informations that the shipyard needs to have in order to decide whether to spawn a ship or not, we find:

1. Current number of ships `N`;
2. Number of turns to the end `t_left` (in this way its flexible about the total number of turns);
3. Halite available `h_tot`;
4. Position of the ships in the map or some variable connected to it (for example we don't want to spawn a ship while another one is dropping the halite in the shipyard, because it would cause a shipwreck);
5. If more map size are used, one could introduce the number of cells of the map `n_cells` as an ulterior information.

### More on the position of the ships variable

We now analize more in detail how to build a variable that can contain the information about how busy is the area around the shipyard, so that the shipyard can decide not to spawn a ship when it is too busy and thus risky to do it.
First of all, given the symmetry of the map and the fact that the action of the shipyard preserves all those symmetries (in fact the new ship is born in the shipyard cell, that is at the center of the map), this variable should assume the same value under equivalent configurations of the system.
Second, it should be local, since the shipyard it is not much concerned about all what is happening in the map (that is the ship agent job).
A minimal approach could be to define an area within a certain number of moves (e.g. 1 or 2) and simply count the number of ships in that area. This is a measure of the concentration of ships and consequently of the risk in spawning a new ship and seeing it crashing with another one. For the first implementation we consider an area whithin a single move from the shipyard for the counting and call the variable so obtained `near_ships`.

## Value estimation

Since the number of variables describing the shipyard state is low and the choice it has to do is binary, it seems natural to use a function instead that a table to approximate the Q value of the choices. In other words, instead of keeping record in a table for each state s and action a of the Q value Q(s,a), we can use a function $\phi$ with the interpretation $\phi(s) = Q(s,1)$, where for convention we use $a=1$ for spawning a ship and $a=0$ for staying still.
Then we have to reformulate the task of the shipyard in order to understand how to determine the function $\phi$.
Notice that this passage from a tabular record of all the states and actions to a more functional approach to the problem assumes that the dependence on each of the variables is continuous (if the function considered is continuous itself). 

### Critical aspects

1. The Q-learning is based on a sort of chain rule to estimate the Q-values, starting from some known value (for example that of the terminal state) and propagating backwards that knowledge through the individual rewards of each state-action that forms the chain of an episode. In this particular situation it's not clear how to assign a step by step reward and would be better to find another approach to disentangle the dependence of the total reward on each single action (taken in a given state).

A possible solution to this problem could be the following:
1. Assume that the null action of the shipyard has always reward 0;
2. Assume that exists a function $f$ continuous in each variable and with some parameters $\underline{w}$ in which the function is linear;

Then we can write the total score $R_{ep}$ of the episode (that more in concrete is the difference between the final and the initial amount of halite) as $R_{ep} = \sum_{i=1}^{n_{ep}}f(s_i|\underline{w})$, where $n_{ep}$ is the total number of spawned ships during that episode. From the linearity of $f$ in $\underline{w}$ we can rewrite this expression as
$$R_{ep} = \underline{w} \cdot \left (\sum_{i=1}^{n_{ep}}\underline{\tilde{s_i}} \right) =  \underline{w} \cdot \underline{\tilde{S}_{ep}}$$ where $\underline{\tilde{s_i}} = \psi(s_i)$ denotes a vectorial representation of the state $s_i$ through some kind of transformation $\psi$ and $\underline{\tilde{S}_{ep}}$ is a compact way to denote the sum component per component of the $\underline{\tilde{s_i}}$ of a given episode. Of course $\underline{w}$ will depend on the transformation $\psi$ chosen.

Now our goal is to learn the weights $\underline{w}$ in order to accurately predict the reward that spawning a ship in a given state of the game will bring. This is a (regression) task that can be stated as follows: given $\left\{\left(\underline{\tilde{S}_{j}},R_j \right)\right\}_{j=1,\ldots,m}$ find the $\underline{w}$ that best fits the data (e.g. minimizing the mean square error).

Notice that we didn't make any assumption on the transformation $\psi$, thus we can use it to build high-level features from the basic features considered in the state representation in order to capture correlations and interactions among the basic features (this can't be done with the scalar product with $\underline{w}$). Of course if $\psi$ was the identity transformation, we could not consider those potential correlations and interactions among variables. A simple yet powerful choice could be that of considering a model of two-feature interaction, that is given by a quadratic "metric" $\underline{(1,s_i)}^T M \underline{(1,s_i)}$. It is simple to see that this is equivalent in our case to choose a polynomial mapping of the second order for the transformation $\psi$ and then applying the scalar product with $\underline{w}$ (also the two cases have the same degrees of freedom, since the metric $M$ is symmetric). This is exactly what we are going to try.

## Shipyard reinforcement learning 

Even though the value estimation procedure seems like a supervised learning problem, this is definitively not the case, because we need to create these samples through interaction with the environment and to do so we also need to have a much earlier feedback than in the supervised learning case, and much more frequently.Furthermore our goal is not to predict the score of an episode knowing what the shipyard has done in that episode, but to teach it how to behave in order to optimize the final reward. 

The structure of the iterative procedure of learning can be break up as follows:
1. Sample a batch of episodes with a policy that spawns a ship every time that thinks it is convenient to do so, i.e. if $f(s_i|\underline{w}) = \underline{w} \cdot \underline{\tilde{s_i}} > 0$, and stays still the other times;
2. Build the episode state representation $\underline{\tilde{S}_{ep}}$;
3. Make a linear regression to estimate the weights $\underline{w}$;
4. Use the weights to update the predictions upon which the policy is based and repeat from 1.

# Heterogeneous learning

The last main problem that we face is to have two different classes of agents bootstrapping themselves at the same time. Moreover the learning of the shipyard strongly depends on how reliable and optimized is the behaviour of the ship and its behaviour changes from the case of a single unit to that of many units, mainly because it has to overcome the problem of crashing against other ships.


## Bootstrapping criticalities

What we can expect from the initial random policies (of both the classes) is that the shipyard will spawn a lot of ships (basically it has a 50% probability of attempting to spawn a ship at each turn, constrained only by the fact that if it has 1000 or less of halite stored the action will be prohibited) and they will often crush into each other. This will lead to a really poor reward for both agents, probably negative if the majority of the ships crashes. It is not clear a priori if for the shipyard receiving only negative rewards will imply learning a function that is always negative and consequently never spawning a ship; if this was the case, it would be a serious bug of the shipyard implementation. However, understanding this issue also brings in a possible solution (not without risk, because it doesn't remove the problem, but just shifts us to a better scenario). This would be to train the ships while the shipyard follows a random policy and, once they start to be successful enough to avoid most of the collisions, stop training the ships and start training the shipyard. Once this first critical phase is completed successfully the whole procedure can be iterated more times, until some sort of convergence in both the learning processes is reached.

The other problem is that the spawning of a ship can't be foreseen by the ships that are going to deposit the halite, hence there is a small but significant probability of shipwrecking while attempting to do the right thing; this of course would lead to a decrease in the Q value of the action of depositing halite from the states around the shipyard. Consequently it must be checked that the tradeoff between the reward in depositing successfully the halite, the risk of shipwrecking with the following penalty and the cost of waiting without doing anything useful are well balanced, so that the risk is affordable and the game doesn't stallmate. If this problem occurs, the simplest thing to try out is to increase the "cost" of time by increasing the baseline penalty that the agent receives at each turn (that basically says that doing nothing is not affordable). The drawback of this solution could be a bias in the weighting of the shipwreck penalty, possibly leading to a sub-optimal policy. However, once we made it out of the first cycle with both the ship and the shipyard agent learning successfully the basics of the game, the policy will still be modified, making room for better policies once the shipyard becomes more reliable in spawning ships.

The alternative, if the double bootstrapping fails, is to program a good starting policy for the shipyard (e.g. spawn a ship every 10 turns and only if the neighbors of the shipyard are empty) and use it to train the ship agent for the first time and then start to train the shipyard with ship agents of which skills we are sure of. But of course this sounds a bit like cheating!

To be more concrete, the schedule could be:
1. Train for 100 epochs of 10 batches of episodes each the ship agent with epsilon greedy policy starting from 0.5 and exponentially decaying to 0.005.
2. Stop updating the ship Q-values (to disentangle the two processes) and start to train the shipyard agent for 5 epochs of 100 batches.
3. Repeat for `N_CYCLES` times (something around 3 - 5), always restarting from $\epsilon = 0.5$ for the ship policy.

This ideal numbers are thought as follows:
- for the ships probably it is better to have more epochs than batches because in this way epsilon decreases more gradually from 0.5 to 0.005;
- for the shipyard a good amount of data is needed before trying to make a regression and estimate the $\underline{w}$; in fact usually regression is a one-shot procedure, not an iterative one. The 5 epochs in this sense are thought as the smallest number for which we can visualize a positive and stable trend in the reward and the preference is given to have reliable metrics than a lot of them.
- `N_CYCLES` should have been as large as possible if we had a disposal infinite time and resources, but of course when the training time magnitude becomes of the order of the day, it is better to leave finish with a good result than waiting for weeks for an optimal one.

# Shipyard implementation

In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../Environment/")
import halite_env as Env
from sklearn.preprocessing import PolynomialFeatures
import encode as cod

In [2]:
from importlib import reload
reload(Env)

<module 'halite_env' from '../Environment/halite_env.py'>

In [3]:
num_players = 1
map_size = 7 # 7 x 7 map
env = Env.HaliteEnv(num_players, map_size)

In [4]:
action_matrix = np.full((map_size,map_size), -1) 
shipyard_action = 1 
state, h, finish, _ = env.step(action_matrix, makeship = shipyard_action)
# s_0 -> map_halite, s_1 -> ship_position, s_2 -> cargo_halite, s_3 -> shipyard_position (not used)
#map_halite = state[:,:,0]
#ship_pos_matrix = state[:,:,1]
#shipy_pos_matrix = state[:,:,3]
print("Ships layer: \n", state[:,:,1])
print("Halite:", h[0][0])

Ships layer: 
 [[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
Halite: 4000.0


In [5]:
def predict_reward(weights, state, h_tot, steps, tot_turns = 400, map_size = 7):
    
    from sklearn.preprocessing import PolynomialFeatures # check if needed
    import encode as cod # check if needed
    
    # h_tot is the halite available
    N = np.count_nonzero(state[:,:,1]) # number of ships in the map
    t_left = tot_turns - steps # number of turns left until the end of the episode

    shipy_enc = cod.one_to_index(state[:,:,3], map_size) 
    shipy_dec = cod.decode(shipy_enc, map_size)
    s1 = shipy_dec + [1,0]
    s2 = shipy_dec + [-1,0]
    s3 = shipy_dec + [0,1]
    s4 = shipy_dec + [0,-1]
    
    s = [shipy_dec,s1,s2,s3,s4]
    mask = np.zeros((map_size,map_size)).astype(int)
    
    for x in s:
        mask[tuple(x)] = 1

    mask = mask.astype(bool)
    near_ships = state[:,:,1][mask].sum() #number of ships that in one move can go to the shipyard
    
    shipy_state = np.array([N,t_left,h_tot,near_ships]) # vectorial state of the four variables
    # [a,b] gets mapped to [1,a,b,a^2,ab,b^2]
    poly = PolynomialFeatures(2)
    poly_state = poly.fit_transform(shipy_state.reshape(1,-1))[0]
    
    # returns the scalar product between the weights and the polynomial mapping of degree 2 of the shipy_state
    return np.dot(weights,poly_state)

## Memorizing trajectories

In order to learn, we must save for each episode all the (N,t_left,h_tot,near_ships) tuples, one for each time a ship was spawn. Of course this number can change from episode to episode, up to a maximum of 400 tuples. At the end of the episode, also the difference between the initial and the final amount of halite must be saved associated to the episode trajectory. Since we want to learn from 100 episodes at least (we could consider also to train on all the data from all the epochs, up to $n\_epochs\times100$ episodes) the simplest container for all this experience is a list of tuples, each one containing a list (or a numpy array) of all the states (N,t_left,h_tot,near_ships) of the episode in one entry and the reward obtained in the other entry.

From that we will build the associated polynomial states $\underline{\tilde{s}_i}$ and also sum feature by feature all those states in an episode in order to form $\underline{\tilde{S}_j}$, that will be used togheter with $R_j$ to estimate $\underline{w}$.

# TODO LIST

1. Implement the new state encoding for multiple agents;
2. Implement shipyard policy from `predict_reward`;
3. Try to run the multiple agents training for a random policy of the shipyard;
4. Implement the framework to train the shipyard (memory buffer + regression);
5. Use some higher level interface for the learning of the two classes, that allows more clarity and concision and also set up things so that the training can be done with continuity through multiple sessions (because it is likely that it will take several days to get a good result);
6. Tune the learning parameters (epochs, batches, learning rate and so on...);
7. Do a time profiling of the learning procedure to understand where is the bottleneck;
8. Optimize all the implementations by tackilng the bottlenecks.

In [6]:
def scalar_to_matrix_action(actions, state, map_size = 7):
    # first get the decoded position of the ship
    ship_pos_matrix = state[:,:,1]
    ships_pos_enc = cod.one_to_index(ship_pos_matrix, map_size)
    # then fill a matrix of -1
    mat_action = np.full((map_size,map_size), -1)
    for i in range(len(ships_pos_enc)):
        pos_dec = cod.decode(ships_pos_enc[i], map_size)
        print("pos_dec: ", pos_dec)
        # finally insert the action in the pos_dec entry
        mat_action[tuple(pos_dec)] = actions[i]
        
    return mat_action

In [14]:
num_players = 1
map_size = 7 # 7 x 7 map
env = Env.HaliteEnv(num_players, map_size)

In [15]:
action_matrix = np.full((map_size,map_size), -1) 
shipyard_action = 1 
state, h, finish, _ = env.step(action_matrix, makeship = shipyard_action)
# s_0 -> map_halite, s_1 -> ship_position, s_2 -> cargo_halite, s_3 -> shipyard_position (not used)
print("Ships layer: \n", state[:,:,1])
print("Halite:", h[0][0])

Ships layer: 
 [[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
Halite: 4000.0


In [16]:
a = [1]
action_matrix = scalar_to_matrix_action(a, state)
print("Action matrix: \n", action_matrix)
shipyard_action = 1 # qui non crea la barca
state, h, finish, _ = env.step(action_matrix, makeship = shipyard_action)
print("Ships layer: \n", state[:,:,1])
print("Halite:", h[0][0])

pos_dec:  [3 3]
Action matrix: 
 [[-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1  1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]]
Ships layer: 
 [[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
Halite: 4000.0


In [17]:
a = [2] #[2]
action_matrix = scalar_to_matrix_action(a, state)
print("Action matrix: \n", action_matrix)
shipyard_action = 1
state, h, finish, _ = env.step(action_matrix, makeship = shipyard_action)
print("Ships layer: \n", state[:,:,1])
print("Halite:", h[0][0])

pos_dec:  [4 3]
Action matrix: 
 [[-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1  2 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]]


ValueError: shape mismatch: value array of shape (2,) could not be broadcast to indexing result of shape (1,)

In [124]:
a = [1,0]
action_matrix = scalar_to_matrix_action(a, state)
print("Action matrix: \n", action_matrix)
shipyard_action = 0
state, h, finish, _ = env.step(action_matrix, makeship = shipyard_action)
print("Ships layer: \n", state[:,:,1])
print("Halite:", h[0][0])

pos_dec:  [3 3]
pos_dec:  [5 3]
Action matrix: 
 [[-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1  1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1  0 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]]
Ships layer: 
 [[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0]]
Halite: 3000.0


Now that we have created a state with which we can work, we try to implement the safest_dir.

In [183]:
def safest_dir(pos_enc, state, map_size = 7):
    # pos_enc is of a single ship
    
    ship_pos_matrix = state[:,:,1]
    shipy_enc = cod.one_to_index(state[:,:,3], map_size) 
    shipy_dec = cod.decode(shipy_enc, map_size)
    pos_dec = cod.decode(pos_enc, map_size)
    shift = (shipy_dec[0]-pos_dec[0],shipy_dec[1]-pos_dec[1])
    centered = np.roll(ship_pos_matrix , shift = shift, axis = (0,1)) #centers map_halite on the ship
    
    s1 = shipy_dec + [0,1]
    s2 = shipy_dec + [0,-1]
    s3 = shipy_dec + [1,0]
    s4 = shipy_dec + [-1,0]
    s = [s1,s2,s3,s4]
    
    mask = np.zeros((map_size,map_size)).astype(int)
    for x in s:
        mask[tuple(x)] = 1

    mask = mask.astype(bool)
    near_ships = centered[mask] # N,W,E,S -> 2,4,3,1
    x = np.array([2,4,3,1])
    if near_ships.sum() < 4:
        safe_dirs = x[~near_ships.astype(bool)] # safe directions
        safest_dir = np.random.choice(safe_dirs)
    else:
        safest_dir = 0
        
    return safest_dir

In [184]:
print(state[:,:,1])
#position_encoded of the ship
ships_pos_enc = cod.one_to_index(state[:,:,1], map_size) # works for multiple ships
print("Encoded position of the ship: ", ships_pos_enc)

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0]]
Encoded position of the ship:  [31 38]


In [225]:
for i in range(len(ships_pos_enc)):
    print(safest_dir(ships_pos_enc[i], state))

4
4


In [226]:
def encode4D(v_dec, L1, L2, L3, L4):
    # v_dec = [v1,v2,v3,v4]
    # returns the encoded version V[v1,v2,v3,v4] of V = np.arange(0,L1*L2*L3*L4)
    V = np.arange(0,L1*L2*L3*L4).reshape((L1,L2,L3,L4))
    v_enc = V[tuple(v_dec)] 
    return v_enc

def decode4D(v_enc, L1, L2, L3, L4):
    # v_enc = V[v1,v2,v3,v4] 
    # V = np.arange(0,L1*L2*L3*L4)
    # returns the decoded version v_dec = [v1,v2,v3,v4] of V[v1,v2,v3,v4] 
    V = np.arange(0,L1*L2*L3*L4).reshape((L1,L2,L3,L4))
    v_dec = np.array([np.where(v_enc == V)[0][0],np.where(v_enc == V)[1][0], 
                      np.where(v_enc == V)[2][0], np.where(v_enc == V)[3][0]])
    return v_dec

In [227]:
# next step is to rewrite the function encode_state for multiple ships